In [21]:
import pandas as pd
import numpy as np
import pdb
import time
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import lightgbm as gbm 

In [22]:
data = pd.read_csv('TrainDataScaledPCA.csv', index_col=0)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,default_ind
application_key,,,,,,,,,,,,,,,,,,,,,
230032,7.696114,12.350336,-2.147227,2.495844,-4.748575,-2.381941,-4.644673,0.773259,-1.294904,1.278156,...,0.279910,-0.345718,0.113600,0.172868,0.324652,0.390063,-0.425547,-0.468056,-0.097686,0.0
230033,-5.545660,-2.045987,-4.134448,0.614403,1.725154,-0.641462,-1.716193,-0.838565,-1.208660,0.701930,...,-0.303886,-0.067337,-0.304120,0.423952,-0.186540,-0.484241,-0.008021,0.345614,-0.319914,1.0
230034,-5.525988,-2.312766,-1.993475,1.496010,-2.269309,1.707946,-1.655883,1.098356,0.146529,-0.700801,...,-0.290246,0.392103,0.135142,-1.019374,-0.270207,-0.572938,0.129733,0.276641,-0.015634,1.0
230035,-5.967580,-2.611269,-1.989661,1.968232,-0.433192,0.345623,-1.086218,1.180024,-1.332320,-0.355089,...,-0.036557,-0.185665,0.158475,0.303761,-0.186965,-0.086659,-0.155855,0.148110,0.003742,0.0
230036,-6.760282,-3.222255,-2.508281,3.129283,0.563894,0.820464,-1.270672,1.217419,-0.440856,1.105332,...,0.201644,0.327657,-0.169663,-0.053480,-0.469242,-0.257605,-0.497360,0.469798,-0.096623,0.0


In [23]:
X = data.drop('default_ind', axis=1)  
y = data['default_ind']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_data = pd.DataFrame.copy(X_train)
train_data['default_ind']  = y_train 

In [ ]:
models_XGB = []
models_LB = []

ns = [50, 100, 200, 300, 400, 500]
max_depths = [1, 2, 3, 5, 10, 15]
ts = [0.45, 0.5, 0.55, 0.6]

i = 0

pos_class = train_data[train_data['default_ind']==1]
neg_class = train_data[train_data['default_ind']==0]
neg_resampled = neg_class.sample(n= int(1 * len(pos_class)), replace=False)

train_data_resampled = pd.concat([pos_class, neg_resampled])

# Use these for Trainings

X_train_resampled = train_data_resampled.drop('default_ind', axis=1)
y_train_resampled = train_data_resampled['default_ind']    


for t in ts: 
    for n in ns:
        for max_depth in max_depths:

            print('\n\n', round(i/144*100, 3), '% Done')
            i += 1
            print('t =', t, 'Number of Estimators =', n, 'Depth =', max_depth)

            #Training

            #XGBoost
            model_XGB = XGBClassifier(n_estimators = n , learning_rate=0.1, max_depth=max_depth ,reg_lambda=0.01, n_jobs = -1, verbose =100) 
            model_XGB.fit(X_train_resampled, y_train_resampled)

            y_hat_XGB = model_XGB.predict(X_train_resampled)
            print('\nTrain Score XGB =', accuracy_score(y_train_resampled, y_hat_XGB))
            print('Train F1 Score XGB =', f1_score(y_train_resampled, y_hat_XGB))    
            print('Train Precision XGB=', precision_score(y_train_resampled, y_hat_XGB))
            print('Train Recall XGB=', recall_score(y_train_resampled, y_hat_XGB))


        #     #AdaBoost
        #     model_AB = AdaBoostClassifier(n_estimators = ns[model_no], learning_rate=0.1, base_estimator=DecisionTreeClassifier(max_depth=5)) 
        #     model_AB.fit(X_train_resampled, y_train_resampled)  

        #     y_hat_AB = model_AB.predict(X_train_resampled)
        #     print('Train Score AB =', accuracy_score(y_train_resampled, y_hat_AB))
        #     print('Train F1 Score AB=', f1_score(y_train_resampled, y_hat_AB))
        #     print('Train Precision AB=' precision_score(y_train_resampled, y_hat_AB))
        #     print('Train Recall AB=' recall_score(y_train_resampled, y_hat_AB))

            #LightBoost
            cv_params = {
            'max_depth': max_depth,
            'objective': 'binary',
            'metric':'auc',  
            'feature_fraction': 1, 
            'bagging_fraction': 0.75,
            'reg_lambda': 1,
            'n_estimators': n
            }

            gbm_train = gbm.Dataset(X_train_resampled, y_train_resampled)
            model_LB = gbm.train(cv_params,  
                        gbm_train, 
                        num_boost_round=900,
                        verbose_eval=1)

            y_train_prob = model_LB.predict(X_train_resampled)
            thres = t
            y_pred_train = np.zeros(len(y_train_prob))
            y_pred_train[np.argwhere(y_train_prob>thres)] = 1
            print('\nTrain Score LB =', accuracy_score(y_train_resampled, y_pred_train))
            print('Train F1 Score LB =', f1_score(y_train_resampled, y_pred_train))
            print('Train Precision LB=', precision_score(y_train_resampled, y_pred_train))
            print('Train Recall LB=', recall_score(y_train_resampled, y_pred_train))

            #Test Scores, accuracy

            y_test_predXGB = model_XGB.predict(X_test)
            print('\nTest Score XGB =', accuracy_score(y_test, y_test_predXGB))
            print('Test F1 Score XGB =', f1_score(y_test, y_test_predXGB)) 
            print('Test Precision XGB=', precision_score(y_test, y_test_predXGB))
            print('Test Recall XGB=', recall_score(y_test, y_test_predXGB))


        #     y_test_predAB = model_AB.predict(X_test)
        #     print('Test Score AB =', accuracy_score(y_test, y_test_predAB))
        #     print('Test F1 Score AB =', f1_score(y_test, y_test_predAB))

            y_test_prob = model_LB.predict(X_test)    
            y_pred_test = np.zeros(len(y_test_prob))
            y_pred_test[np.argwhere(y_test_prob>thres)] = 1    
            print('\nTest Score LB=', accuracy_score(y_test, y_pred_test))
            print('Test F1 Score LB=', f1_score(y_test, y_pred_test))
            print('Test Precision LB=', precision_score(y_test, y_pred_test))
            print('Test Recall LB=', recall_score(y_test, y_pred_test))


            models_XGB.append(model_XGB)
            models_LB.append(model_LB)




 0.0 % Done
t = 0.45 Number of Estimators = 50 Depth = 1

Train Score XGB = 0.684547765303961
Train F1 Score XGB = 0.7166841053933226
Train Precision XGB= 0.650422997237569
Train Recall XGB= 0.7979771234907859

Train Score LB = 0.6737714467273883
Train F1 Score LB = 0.718705052396064
Train Precision LB= 0.631697234819601
Train Recall LB= 0.8335098496081339

Test Score XGB = 0.6380472402695975
Test F1 Score XGB = 0.5602360752489857
Test Precision XGB= 0.4315263098079327
Test Recall XGB= 0.7983599663582843

Test Score LB= 0.6055619649037586
Test F1 Score LB= 0.5493270431524906
Test Precision LB= 0.40991923793746116
Test Recall LB= 0.8324222035323802


 0.694 % Done
t = 0.45 Number of Estimators = 50 Depth = 2

Train Score XGB = 0.7027907223045965
Train F1 Score XGB = 0.7203189236327394
Train Precision XGB= 0.6802032845513152
Train Recall XGB= 0.765462825672527

Train Score LB = 0.6929675916119467
Train F1 Score LB = 0.7259665374799129
Train Precision LB= 0.6555138272447935
Train Recall